<a href="https://colab.research.google.com/github/ryukya/AI_2020/blob/SVM/303_data_13_feature_Mushroom_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data preparation and resampling

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np

##load data
df_mushroom = pd.read_csv('mushrooms.csv')
df_mushroom=df_mushroom.sample(n=303, random_state=10)



##transform value
lb=LabelEncoder()
df_mushroom=df_mushroom.apply(lb.fit_transform)
datas=df_mushroom.rename(columns={ "class": "lab"}, errors="raise")

datas.head()


,lab,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
582,0,4,2,7,1,3,1,0,0,4,0,2,2,2,7,7,0,2,1,4,2,2,1
2288,1,1,1,4,1,6,1,0,1,7,0,3,2,2,7,7,0,2,1,4,2,3,5
1789,0,4,1,7,0,5,1,1,0,5,1,3,0,2,7,7,0,2,1,0,2,3,1
3374,0,1,2,2,1,5,1,0,0,9,1,1,2,2,7,6,0,2,1,4,3,4,0
814,1,4,1,4,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,2,4,5


In [2]:
##separate label and features data
dt_label = datas['lab']
df_mushroom = datas.drop(['lab'],axis=1)
##put data to Y 
Y = []
for val in dt_label:
    if(val == 1): ## val=1 poisonus
        Y.append(-1)
    else:
        Y.append(1)
print(df_mushroom.head())

## reduce features to 13 
from random import seed
from random import randint
seed(1)
cols = []
col_tot= len(df_mushroom.columns)

col_del=col_tot-12

for _ in range(col_del):
	value = randint(0, col_tot)
	cols.append(value)
print (cols)
df_mushroom.drop(df_mushroom.columns[cols],axis=1,inplace=True)
print(df_mushroom.head())



      cap-shape  cap-surface  cap-color  ...  spore-print-color  population  habitat
582           4            2          7  ...                  2           2        1
2288          1            1          4  ...                  2           3        5
1789          4            1          7  ...                  2           3        1
3374          1            2          2  ...                  3           4        0
814           4            1          4  ...                  2           4        5

[5 rows x 22 columns]
[4, 18, 2, 8, 3, 15, 14, 15, 20, 12]
      cap-shape  cap-surface  ...  spore-print-color  habitat
582           4            2  ...                  2        1
2288          1            1  ...                  2        5
1789          4            1  ...                  2        1
3374          1            2  ...                  3        0
814           4            1  ...                  2        5

[5 rows x 13 columns]


In [3]:
##put data to X
X = df_mushroom.values.tolist()

## prepare variable
x_train = []
y_train = []
x_test = []
y_test = []

##split data
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=10)

##data leng for  data training and testing
lengt=len(df_mushroom)
train_len= int(0.8*lengt)
test_len=int( 0.2*lengt)+1

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

y_train = y_train.reshape(train_len,1)
y_test = y_test.reshape(test_len,1)

print("length: ",lengt,"---- training:",train_len,"----test:",test_len)



length:  303 ---- training: 242 ----test: 61




##Evaluation Function

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
def evaluate (y_test,predictions):
  print ("Accuracy Score")
  print(accuracy_score(y_test,predictions))
  print ("Confusion Matrix")
  print(confusion_matrix(y_test, predictions))
  print ("Classification report")
  print(classification_report(y_test, predictions))

##SVM Function

In [0]:
def svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc):
  training=[]
  ##train_fx & training reshape 
  for col in range(x_train.shape[1]):
    training.append(x_train[:,col].reshape(train_len,1))
  ##make w array (zeros) 
  w_zeros=[]
  for x in training:
    w_zeros.append(np.zeros((train_len,1)))
  epochs = 1
  alpha = alpa
  while(epochs < epoc):
    ##sum of array in w_zeros*array in training
    sumval = None 
    for xw,yt in zip(w_zeros, training):
      xy=xw*yt
      if sumval is None:
       y1=xy
      else:
        y1= y1 + xy
      sumval =y1 
    prod = sumval * y_train
    count = 0
    for val in prod:
      if(val >= 1):
        cost = 0
        for i in range(len(w_zeros)):
         w_zeros[i]= w_zeros[i] - alpha * (2 * 1/epochs * w_zeros[i])
      else:
        cost = 1 - val
        for i in range(len(w_zeros)):
         w_zeros[i]= w_zeros[i] + alpha * (training[i][count] * y_train[count] - 2 * 1/epochs * w_zeros[i])
      count += 1
    epochs += 1
  ## Clipping weights  and reshape array
  index = list(range(test_len,train_len))
  for i in range(len(w_zeros)):
    w_zeros[i]=np.delete(w_zeros[i],index)
    w_zeros[i] = w_zeros[i].reshape(test_len,1)
  ## Extract the test data features 
  testing=[]
  for col in range(x_test.shape[1]):
    testing.append(x_test[:,col].reshape(test_len,1))
  ## Predict
  pred_val = None 
  for xw,yt in zip(w_zeros, testing):
    xy=xw*yt
    if pred_val is None:
      y1=xy
    else:
      y1= y1 + xy
    pred_val =y1 
  #add prediction result into list
  predictions = []
  for val in pred_val:
      if(val > 1):
          predictions.append(1)
      else:
          predictions.append(-1)
  evaluate(y_test,predictions)     



##SVM with alpha= 0.002 and epoch =500

In [6]:
alpa=0.002
epoc=500
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.45901639344262296
Confusion Matrix
[[25  0]
 [33  3]]
Classification report
              precision    recall  f1-score   support

          -1       0.43      1.00      0.60        25
           1       1.00      0.08      0.15        36

    accuracy                           0.46        61
   macro avg       0.72      0.54      0.38        61
weighted avg       0.77      0.46      0.34        61



##SVM with alpha= 0.001 and epoch =1000

In [7]:
alpa=0.001
epoc=1000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.9016393442622951
Confusion Matrix
[[20  5]
 [ 1 35]]
Classification report
              precision    recall  f1-score   support

          -1       0.95      0.80      0.87        25
           1       0.88      0.97      0.92        36

    accuracy                           0.90        61
   macro avg       0.91      0.89      0.90        61
weighted avg       0.91      0.90      0.90        61



##SVM with alpha= 0.0005 and epoch =2000

In [8]:
alpa=0.0005
epoc=2000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.9016393442622951
Confusion Matrix
[[24  1]
 [ 5 31]]
Classification report
              precision    recall  f1-score   support

          -1       0.83      0.96      0.89        25
           1       0.97      0.86      0.91        36

    accuracy                           0.90        61
   macro avg       0.90      0.91      0.90        61
weighted avg       0.91      0.90      0.90        61



##Result if Using Scikit SVM Library

In [9]:
from sklearn import svm
sv_f = svm.SVC(kernel='linear') # Linear Kernel
sv_f.fit(x_train, y_train)
y_pred = sv_f.predict(x_test)
evaluate(y_test, y_pred)


Accuracy Score
0.9344262295081968
Confusion Matrix
[[22  3]
 [ 1 35]]
Classification report
              precision    recall  f1-score   support

          -1       0.96      0.88      0.92        25
           1       0.92      0.97      0.95        36

    accuracy                           0.93        61
   macro avg       0.94      0.93      0.93        61
weighted avg       0.94      0.93      0.93        61



/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
